In [ ]:
#code Accessing Data from Apache Hive
# sample 1

import os
import pandas
from impala.dbapi import connect
from impala.util import as_pandas

# Specify HIVE_HMS_HOST as an environment variable in your project settings
HIVE_HMS_HOST = os.getenv('HIVE_HS2_HOST', '<hiveserver2_hostname>')

# This connection string depends on your cluster setup and authentication mechanism
conn = connect(host=HIVE_HS2_HOST,
               port='10000',
               auth_mechanism='GSSAPI',
               kerberos_service_name='hive')
cursor = conn.cursor()
cursor.execute('SHOW TABLES')
tables = as_pandas(cursor)
tables

In [ ]:
# sample 2
  hive_cur.execute('SELECT * FROM  `my_db`.`my_first_hdfs_based_external_table` ')
  # Fetch ALL table records in the tuple (could be extremely slow for large tables)
  records = hive_cur.fetchall()
  for record in records:
      # Do smth with record
      pass
  # Fetch ONE working like iterator
  record = hive_cur.fetchone()
  while record is not None:
      # Do smth with record
      record = hive_cur.fetchone()

In [ ]:
# sample 3 
import pandas as pd
from pyhive import hive
class HiveConnection:
    @staticmethod
    def select_query(query_str: str, database:str =HIVE_SCHEMA) -> pd.DataFrame:
        """
        Execute a select query which returns a result set
        :param query_str: select query to be executed
        :param database: Hive Schema
        :return:
        """
        conn = hive.Connection(host=HIVE_URL, port=HIVE_PORT, database=database, username=HIVE_USER)

        try:
            result = pd.read_sql(query_str, conn)
            return result
        finally:
            conn.close()

    @staticmethod
    def execute_query(query_str: str, database: str=HIVE_SCHEMA):
        """
        execute an query which does not return a result set.
        ex: INSERT, CREATE, DROP, ALTER TABLE
        :param query_str: Hive query to be executed
        :param database: Hive Schema
        :return:
        """
        conn = hive.Connection(host=HIVE_URL, port=HIVE_PORT, database=database, username=HIVE_USER)
        cur = conn.cursor()
        # Make sure to set the staging default to HDFS to avoid some potential S3 related errors
        cur.execute("SET hive.exec.stagingdir=/tmp/hive/")
        cur.execute("SET hive.exec.scratchdir=/tmp/hive/")
        try:
            cur.execute(query_str)
            return "SUCCESS"
        finally:
            conn.close()